# Nettoyage des données écologiques pour la base Hub Eau

In [1]:
import os
import zipfile
import pandas as pd
import shutil


# Paramètre rivière pour préfixer les fichiers
riviere = "Seine"

zip_path = os.path.abspath(os.path.join(os.getcwd(), "..", f"{riviere}_donnees.zip"))

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Charger chaque CSV dans une variable dédiée
    indic = pd.read_csv(zip_ref.open("Poissons/poissons_indicateurs.csv"))
    obs = pd.read_csv(zip_ref.open("Poissons/poissons_observations.csv"))
    ops = pd.read_csv(zip_ref.open("Poissons/poissons_operations.csv"))
    stats = pd.read_csv(zip_ref.open("Poissons/poissons_stations.csv"))

    analyse_pc = pd.read_csv(zip_ref.open("Qualite des Cours d'Eau/qualite_eau_analyse_pc.csv"))
    cond_env_pc = pd.read_csv(zip_ref.open("Qualite des Cours d'Eau/qualite_eau_condition_env_pc.csv"))
    operation_pc = pd.read_csv(zip_ref.open("Qualite des Cours d'Eau/qualite_eau_operation_pc.csv"))
    station_pc = pd.read_csv(zip_ref.open("Qualite des Cours d'Eau/qualite_eau_station_pc.csv"))

C:\Users\ferna\AppData\Local\Temp\ipykernel_7388\1768735711.py:13: DtypeWarning: Columns (20,41,50,55,56,57,58,60,62,64,65,66,67,68,69,70,85,88,92,93,101,102,103,104,105,106,113) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv(zip_ref.open("Poissons/poissons_observations.csv"))
C:\Users\ferna\AppData\Local\Temp\ipykernel_7388\1768735711.py:17: DtypeWarning: Columns (12,20,41,42,51) have mixed types. Specify dtype option on import or set low_memory=False.
  analyse_pc = pd.read_csv(zip_ref.open("Qualite des Cours d'Eau/qualite_eau_analyse_pc.csv"))


# 1. Nettoyage des données pour les poissons
## 1.1 On regarde les métadonnées communes au 4 tables (on va identifier les jointures primordiales et supprimer les données facultatives/inutiles pour notre étude)

In [2]:
colonnes_indic = set(indic.columns)
colonnes_obs = set(obs.columns)
colonnes_ops = set(ops.columns)
colonnes_stats = set(stats.columns)

colonnes_communes = colonnes_indic & colonnes_obs & colonnes_ops & colonnes_stats

print("Colonnes communes aux 4 fichiers Poissons :")
for col in sorted(colonnes_communes):
    print(col)


Colonnes communes aux 4 fichiers Poissons :
code_bassin
code_commune
code_departement
code_entite_hydrographique
code_epsg_projection_point_prelevement
code_epsg_projection_station
code_point_prelevement
code_point_prelevement_aspe
code_point_prelevement_wama
code_projection_point_prelevement
code_projection_station
code_region
code_station
codes_dispositifs_collecte
coordonnee_x_point_prelevement
coordonnee_x_station
coordonnee_y_point_prelevement
coordonnee_y_station
geometry
latitude
libelle_bassin
libelle_commune
libelle_departement
libelle_entite_hydrographique
libelle_point_prelevement_wama
libelle_projection_point_prelevement
libelle_projection_station
libelle_region
libelle_station
libelles_dispositifs_collecte
lieu_dit_point_prelevement
localisation_precise_point_prelevement
longitude
objectifs_operation
uri_entite_hydrographique
uri_station
uris_dispositifs_collecte


### Une fois qu'on a identifié les metadata inutiles, on les liste et on les supprime des 4 tables

In [3]:
useless_data = ['code_bassin',
                'code_epsg_projection_point_prelevement',
                'code_epsg_projection_station',
                'code_point_prelevement_wama',
                'code_projection_point_prelevement',
                'code_projection_station',
                'codes_dispositifs_collecte',
                'geometry',
                'libelle_point_prelevement_wama',
                'libelle_projection_point_prelevement',
                'libelle_projection_station',
                'libelles_dispositifs_collecte',
                'lieu_dit_point_prelevement',
                'localisation_precise_point_prelevement',
                'uri_entite_hydrographique',
                'uri_station',
                'uris_dispositifs_collecte']

for df in [obs, ops, stats, indic]:
    cols_to_drop = [col for col in useless_data if col in df.columns]
    df.drop(columns=cols_to_drop, inplace=True)

In [4]:
print(obs.shape)
print(ops.shape)
print(stats.shape)
print(indic.shape)

(80918, 97)
(489, 169)
(39, 51)
(352, 139)


## 1.2 On va maintenant regarder quoi supprimer pour les metadata propres à chaque table

In [5]:
def colonnes_propres(df, nom_df):
    propres = [col for col in df.columns if col not in colonnes_communes]
    print(f"\nColonnes propres à {nom_df} ({len(propres)}):")
    for c in propres:
        print(f" - {c}")
    return propres

### Pour Stations

In [6]:
prop_stats = colonnes_propres(stats, 'stats')


Colonnes propres à stats (31):
 - statut_station
 - date_modification_station
 - localisation_precise_station
 - pk_aval
 - statut_point_prelevement
 - date_modification_point_prelevement_aspe
 - code_support
 - libelle_support
 - uri_support
 - mnemoniques_dispositifs_collecte
 - code_unite_hydrographique
 - libelle_unite_hydrographique
 - code_masse_eau
 - uri_masse_eau
 - distance_mer
 - distance_maree
 - formation_geologique
 - code_occupation_sol
 - libelle_occupation_sol
 - zone_huet
 - code_regime_hydrologique
 - libelle_regime_hydrologique
 - accessibilite
 - largeur_lit_mineur
 - distance_source
 - altitude
 - pente
 - surface_bassin_versant_amont
 - temperature_janvier
 - temperature_juillet
 - eligibilite_ipr_iprplus


In [7]:
# Colonnes à supprimer dans stats
colonnes_a_supprimer_stats = [
    'mnemoniques_dispositifs_collecte',
    'distance_source',
    'distance_mer',
    'distance_maree',
    'accessibilite',
    'uri_support',
    'uri_masse_eau',
    'date_modification_point_prelevement_aspe',
    'code_regime_hydrologique',
    'libelle_regime_hydrologique',
    'temperature_juillet',
    'temperature_janvier',
    'altitude',
    'pente',
    'surface_bassin_versant_amont',
    'largeur_lit_mineur',  
    'code_occupation_sol',
    'libelle_occupation_sol',
    'zone_huet'
]

# Supprimer ces colonnes dans stats (si elles existent)
stats_clean = stats.drop(columns=[col for col in colonnes_a_supprimer_stats if col in stats.columns])

### Pour Opérations

In [8]:
prop_ops = colonnes_propres(ops, 'ops')


Colonnes propres à ops (149):
 - code_operation
 - date_operation
 - etat_avancement_operation
 - code_qualification_operation
 - libelle_qualification_operation
 - nombre_traits
 - nombre_points_anguilles
 - protocole_peche
 - moyen_prospection
 - operateur_code
 - operateur_libelle
 - operateur_libelle_aspe
 - uri_operateur
 - commanditaire_code
 - commanditaire_libelle
 - commanditaire_libelle_aspe
 - uri_commanditaire
 - expert_technique_code
 - expert_technique_libelle
 - expert_technique_libelle_aspe
 - uri_expert_technique
 - date_creation_operation
 - date_modification_operation
 - pente_ligne_eau
 - section_mouillee
 - durete_totale
 - temperature_max_eau
 - niveau_typologique_theorique
 - code_categorie_piscicole
 - libelle_categorie_piscicole
 - temperature_air_bassin_versant
 - precipitation_bassin_versant
 - amplitude_thermique_air_station
 - temperature_air_station
 - espece_ciblee
 - espece_ciblee_code_taxon
 - espece_ciblee_code_alternatif_taxon
 - espece_ciblee_nom_co

In [9]:
colonnes_a_supprimer_ops = [
    # Opérateurs non essentiels
    'uri_operateur',
    'commanditaire_code',
    'commanditaire_libelle',
    'commanditaire_libelle_aspe',
    'uri_commanditaire',
    'expert_technique_code',
    'expert_technique_libelle',
    'expert_technique_libelle_aspe',
    'uri_expert_technique',
    
    # Dates création et modification (peut être utile, à virer si tu veux)
    # 'date_creation_operation',
    # 'date_modification_operation',
    
    # Paramètres très techniques hydromorphologie
    'pente_ligne_eau',
    'section_mouillee',
    'isolement_aval',
    'isolement_amont',
    
    # Matériel et pêche
    'fabricant_materiel',
    'modele_materiel',
    'tension',
    'intensite',
    'puissance',
    'nombre_anodes',
    'nombre_epuisettes',
    'maille_epuisette',
    
    # Hydrologie détaillée
    'code_tendance_debit',
    'libelle_tendance_debit',
    'code_turbidite',
    'libelle_turbidite',
    'code_conditions_hydrologiques',
    'libelle_conditions_hydrologiques',
    'debit_horaire',
    
    # Faciès, granulo, végétation très détaillés
    'facies_code_type',
    'facies_libelle_type',
    'facies_profondeur_moyenne',
    'facies_importance_relative',
    'facies_code_granulo_dominante',
    'facies_libelle_granulo_dominante',
    'facies_code_granulo_accessoire',
    'facies_libelle_granulo_accessoire',
    'facies_code_type_colmatage',
    'facies_libelle_type_colmatage',
    'facies_vegetation_dominante',
    'facies_pourcentage_recouvrement_vegetation',
    
    # Ombre, colmatage, végétation abondance détaillée
    'code_intensite_ombrage',
    'libelle_intensite_ombrage',
    'code_intensite_ombrage_riviere',
    'libelle_intensite_ombrage_riviere',
    'code_intensite_colmatage',
    'libelle_intensite_colmatage',
    'vegetation_dominante',
    'code_granulo_donnees_env_dominante',
    'libelle_granulo_donnees_env_dominante',
    'code_granulo_donnees_env_accessoire',
    'libelle_granulo_donnees_env_accessoire',
    'code_abondance_racine',
    'libelle_abondance_racine',
    'code_abondance_vegetation_aquatique',
    'libelle_abondance_vegetation_aquatique',
    'code_abondance_sous_berges',
    'libelle_abondance_sous_berges',
    'code_abondance_embacles_souches',
    'libelle_abondance_embacles_souches',
    'code_abondance_trous_fosses',
    'libelle_abondance_trous_fosses',
    'code_abondance_vegetation_bordure',
    'libelle_abondance_vegetation_bordure',
    'code_abondance_abris_rocheux',
    'libelle_abondance_abris_rocheux',
    
    # Secteurs et activités humaines
    'presence_ripisylve_equilibree',
    'presence_diversite_ecoulement',
    'pourcentage_plat_lent',
    'pourcentage_plat_courant',
    'pourcentage_courant',
    'presence_secteur_canalise',
    'presence_secteur_navigue',
    'presence_sports_nautiques',
    'presence_curage',
    'presence_faucardage',
    'presence_deboisement',
    'presence_modification_morphologique',
    'presence_extraction_granulats',
    
    # Observations diverses très spécifiques
    'observations_vegetation',
    'observations_colmatage',
    'observations_lits_rives',
    'observations_station',
    'observations_repeuplement',
    
    # Abondances très spécifiques
    'abondance_vegetation_aquatique',
    'code_intensite_ombrage',
    'libelle_intensite_ombrage',
    'code_intensite_colmatage',
    'libelle_intensite_colmatage',
    'abondance_trous_fosses',
    'abondance_sous_berges',
    'abondance_abris_rocheux',
    'abondance_embacles_souches',
    'abondance_abri_vegetal_aquatique',
    'abondance_vegetation_bordure',
    
    # Repeuplement, limites, captures spécifiques
    'presence_repeuplement',
    'operation_sans_poisson',
    'limite_0_plus',
    'limite_1_plus',
    'nb_anguilles_vues_non_capturees',
    
    # Espèces autres
    'autres_especes_codes_taxon',
    'autres_especes_codes_alternatifs_taxon',
    'autres_especes_noms_communs_taxon',
    'autres_especes_noms_latins_taxon',
    
    # Commentaires spécifiques
    'commentaire'
]


# Supprimer ces colonnes dans ops (si elles existent)
ops_clean = ops.drop(columns=[col for col in colonnes_a_supprimer_ops if col in ops.columns])

### Pour Observations

In [10]:
obs_stats = colonnes_propres(obs, 'obs')


Colonnes propres à obs (77):
 - code_operation
 - date_operation
 - etat_avancement_operation
 - code_qualification_operation
 - libelle_qualification_operation
 - protocole_peche
 - code_prelevement_elementaire
 - type_prelevement_elementaire
 - duree_prelevement_elementaire
 - numero_passage
 - type_points
 - nombre_points
 - nombre_points_sans_poisson
 - nombre_points_facies_courant
 - nombre_points_facies_plat
 - nombre_points_facies_profond
 - nombre_points_annexes
 - nombre_points_berge
 - nombre_points_chenal
 - nom_ambiance
 - longueur_ambiance
 - largeur_ambiance
 - profondeur_ambiance
 - code_facies_ambiance
 - libelle_facies_ambiance
 - situation_ambiance
 - localisation_ambiance
 - classe_vitesse_courant_ambiance
 - code_ombrage_ambiance
 - libelle_ombrage_ambiance
 - code_granulo_ambiance_dominante
 - libelle_granulo_ambiance_dominante
 - code_granulo_ambiance_accessoire
 - libelle_granulo_ambiance_accessoire
 - vegetation_ambiance
 - abondance_cache_vegetation_aquatique_

In [11]:
colonnes_a_supprimer_obs = [
    # Infos administratives ou statutaires sans intérêt écologique
    'etat_avancement_operation',      # statut administratif de l'opération
    'code_qualification_operation',  # qualification administrative
    'libelle_qualification_operation',
    'protocole_peche',                # protocole de pêche, trop spécifique

    'numero_passage',                 # numéro de passage technique peu utile

    # Détails très fins sur habitat, peu pertinents pour analyse large
    'nombre_points_facies_courant',
    'nombre_points_facies_plat',
    'nombre_points_facies_profond',
    'nombre_points_annexes',
    'nombre_points_berge',
    'nombre_points_chenal',

    'longueur_ambiance',              # dimensions précises non nécessaires
    'largeur_ambiance',
    'profondeur_ambiance',
    'code_facies_ambiance',           # codes habitat très précis
    'libelle_facies_ambiance',

    'classe_vitesse_courant_ambiance', # détails hydrologiques fins

    'code_granulo_ambiance_accessoire',  # granulométrie accessoire peu utile
    'libelle_granulo_ambiance_accessoire',

    'abondance_cache_granulo_ambiance',  # granulométrie détails peu critiques

    'uri_taxon',                       # URL référentielle non nécessaire

    'methode_estimation_poids_lot',    # méthodes estimation poids peu utiles
    'methode_estimation_poids_individu',

    # Détails pathologies (à virer sauf si étude sanitaire spécifique)
    'uris_fractions_pathologies_lot',
    'uris_fractions_pathologies_individu'
]


# Supprimer ces colonnes dans obs (si elles existent)
obs_clean = obs.drop(columns=[col for col in colonnes_a_supprimer_obs if col in obs.columns])

### Pour Indicateurs

In [12]:
prop_indic = colonnes_propres(indic, 'indic')


Colonnes propres à indic (119):
 - code_operation
 - date_operation
 - etat_avancement_operation
 - code_qualification_operation
 - libelle_qualification_operation
 - protocole_peche
 - ipr_date_execution
 - ipr_version
 - ipr_surface_prospectee
 - ipr_code_unite_hydrographique
 - ipr_superficie_bassin_versant
 - ipr_distance_source
 - ipr_largeur_eau
 - ipr_pente
 - ipr_profondeur
 - ipr_altitude
 - ipr_temperature_air_janvier
 - ipr_temperature_air_juillet
 - ipr_effectif
 - ipr_a
 - ipr_g
 - ipr_v
 - ipr_t1
 - ipr_t2
 - ipr_nte_observe
 - ipr_ner_observe
 - ipr_nel_observe
 - ipr_dit_observe
 - ipr_dii_observe
 - ipr_dio_observe
 - ipr_dti_observe
 - ipr_nte_theorique
 - ipr_ner_theorique
 - ipr_nel_theorique
 - ipr_dit_theorique
 - ipr_dii_theorique
 - ipr_dio_theorique
 - ipr_dti_theorique
 - ipr_nte
 - ipr_ner
 - ipr_nel
 - ipr_dit
 - ipr_dii
 - ipr_dio
 - ipr_dti
 - ipr_note
 - ipr_code_classe
 - ipr_libelle_classe
 - ipr_codes_taxon
 - ipr_codes_alternatifs_taxon
 - ipr_noms_c

In [13]:
colonnes_a_supprimer_indic = [

    # Données administratives et statutaires sans intérêt écologique
    'etat_avancement_operation',
    'code_qualification_operation',
    'libelle_qualification_operation',
    'protocole_peche',
    
    # Dates, versions, doublons inutiles
    'ipr_date_execution',
    'ipr_version',
    'iprplus_date_execution',
    'iprplus_version',
    
    # Informations géographiques et hydrologiques très détaillées / redondantes
    'ipr_code_unite_hydrographique',
    'ipr_superficie_bassin_versant',
    'ipr_distance_source',
    'ipr_pente',
    'ipr_altitude',
    'iprplus_code_formation_geologique',
    'iprplus_code_unite_hydrographique',
    'iprplus_code_regime_hydrologique',
    'iprplus_zone_huet',
    'iprplus_superficie_bassin_versant',
    'iprplus_pente',
    'iprplus_precipitation_bassin_versant',
    'iprplus_amplitude_thermique_air',
    'iprplus_temperature_air_bassin_versant',
    'iprplus_temperature_air_station',
    'iprplus_debit',
    'iprplus_evaporation',
    'iprplus_puissance',
    'iprplus_ric_util',
    'iprplus_run_off',
    'iprplus_seuil_taux_0_plus',
    
    # Colonnes d’abondance très détaillées (théorique et observée)
    # (On peut toujours les réintégrer plus tard si besoin)
    'ipr_nte_observe',
    'ipr_ner_observe',
    'ipr_nel_observe',
    'ipr_dit_observe',
    'ipr_dii_observe',
    'ipr_dio_observe',
    'ipr_dti_observe',
    'ipr_nte_theorique',
    'ipr_ner_theorique',
    'ipr_nel_theorique',
    'ipr_dit_theorique',
    'ipr_dii_theorique',
    'ipr_dio_theorique',
    'ipr_dti_theorique',
    
    
    'iprplus_abondance_rhpar_observe',
    'iprplus_limno_observe',
    'iprplus_lipar_observe',
    'iprplus_omni_observe',
    'iprplus_stther_observe',
    'iprplus_abondance_rhpar_theorique',
    'iprplus_limno_theorique',
    'iprplus_lipar_theorique',
    'iprplus_omni_theorique',
    'iprplus_stther_theorique',
    'iprplus_tol_theorique',
    'iprplus_abondance_rhpar',
    'iprplus_limno',
    'iprplus_lipar',
    'iprplus_omni',
    'iprplus_stther'
]

# Supprimer ces colonnes dans indic (si elles existent)
indic_clean = indic.drop(columns=[col for col in colonnes_a_supprimer_indic if col in indic.columns])

## 1.3 On affiche les tailles des tables avant et après nettoyage

In [14]:
# Affichage de la structure des DataFrames poissons

print("=== poissons_indicateurs ===")
print(f"Taille avant nettoyage : {indic.shape}")
print(f"Taille après nettoyage  : {indic_clean.shape}\n")

print("=== poissons_observations ===")
print(f"Taille avant nettoyage : {obs.shape}")
print(f"Taille après nettoyage  : {obs_clean.shape}\n")

print("=== poissons_operations ===")
print(f"Taille avant nettoyage : {ops.shape}")
print(f"Taille après nettoyage  : {ops_clean.shape}\n")

print("=== poissons_stations ===")
print(f"Taille avant nettoyage : {stats.shape}")
print(f"Taille après nettoyage  : {stats_clean.shape}\n")


=== poissons_indicateurs ===
Taille avant nettoyage : (352, 139)
Taille après nettoyage  : (352, 80)

=== poissons_observations ===
Taille avant nettoyage : (80918, 97)
Taille après nettoyage  : (80918, 72)

=== poissons_operations ===
Taille avant nettoyage : (489, 169)
Taille après nettoyage  : (489, 69)

=== poissons_stations ===
Taille avant nettoyage : (39, 51)
Taille après nettoyage  : (39, 32)



# 2. Nettoyage des données de qualité des cours d'eau

In [15]:
colonnes_analyse_pc = set(analyse_pc.columns)
colonnes_cond_env_pc = set(cond_env_pc.columns)
colonnes_operation_pc = set(operation_pc.columns)
colonnes_station_pc = set(station_pc.columns)

colonnes_communes = colonnes_analyse_pc & colonnes_cond_env_pc & colonnes_operation_pc & colonnes_station_pc

print("Colonnes communes aux 4 fichiers Qualite des Rivières :")
for col in sorted(colonnes_communes):
    print(col)


Colonnes communes aux 4 fichiers Qualite des Rivières :
code_station
geometry
latitude
libelle_station
longitude
uri_station


In [16]:
useless_cols = ['geometry',
               'uri_station'
               ]

for df in [analyse_pc, cond_env_pc, operation_pc, station_pc]:
    cols_to_drop = [col for col in useless_cols if col in df.columns]
    df.drop(columns=cols_to_drop, inplace=True)

In [17]:
prop_station_pc = colonnes_propres(station_pc, 'station_pc')


Colonnes propres à station_pc (38):
 - durete
 - coordonnee_x
 - coordonnee_y
 - code_projection
 - libelle_projection
 - code_commune
 - libelle_commune
 - code_departement
 - libelle_departement
 - code_region
 - libelle_region
 - code_cours_eau
 - nom_cours_eau
 - uri_cours_eau
 - code_masse_deau
 - code_eu_masse_deau
 - nom_masse_deau
 - uri_masse_deau
 - code_eu_sous_bassin
 - nom_sous_bassin
 - code_bassin
 - code_eu_bassin
 - nom_bassin
 - uri_bassin
 - type_entite_hydro
 - commentaire
 - date_creation
 - date_arret
 - date_maj_information
 - finalite
 - localisation_precise
 - nature
 - altitude_point_caracteristique
 - point_kilometrique
 - premier_mois_annee_etiage
 - superficie_bassin_versant_reel
 - superficie_bassin_versant_topo
 - uri_sous_bassin


In [18]:
colonnes_a_supprimer_station_pc = [
    "uri_cours_eau",
    "code_eu_masse_deau",
    "uri_masse_deau",
    "code_eu_sous_bassin",
    "uri_sous_bassin",
    "code_eu_bassin",
    "uri_bassin",
    "code_projection",
    "libelle_projection",
    "coordonnee_x",
    "coordonnee_y",
    "type_entite_hydro",
    "commentaire",
    "finalite",
    "nature",
    "localisation_precise",
    "date_maj_information",
    "superficie_bassin_versant_topo",
    "premier_mois_annee_etiage",
]

# Supprimer ces colonnes dans station_pc (si elles existent)
station_pc_clean = station_pc.drop(columns=[col for col in colonnes_a_supprimer_station_pc if col in station_pc.columns])

In [19]:
prop_operation_pc = colonnes_propres(operation_pc, 'operation_pc')


Colonnes propres à operation_pc (38):
 - x_prelevement
 - y_prelevement
 - code_projection
 - libelle_projection
 - code_support
 - libelle_support
 - uri_support
 - code_methode
 - nom_methode
 - uri_methode
 - date_prelevement
 - date_fin
 - heure_fin
 - code_zone_verticale_prospectee
 - mnemo_zone_verticale_prospectee
 - profondeur
 - code_difficulte
 - mnemo_difficulte
 - code_accreditation
 - mnemo_accreditation
 - agrement
 - code_finalite
 - libelle_finalite
 - commentaires
 - code_reseau
 - nom_reseau
 - uri_reseau
 - code_producteur
 - nom_producteur
 - uri_producteur
 - code_preleveur
 - nom_preleveur
 - uri_preleveur
 - code_operation
 - code_prelevement
 - code_point_eau_surface
 - code_banque_reference
 - heure_prelevement


In [20]:
colonnes_inutiles_operation_pc = [
    # Coordonnées projetées et système de projection (redondant si lat/lon)
    "x_prelevement",
    "y_prelevement",
    "code_projection",
    "libelle_projection",

    # URIs liés aux supports, méthodes, réseaux et producteurs (peu utiles)
    "uri_support",
    "uri_methode",
    "uri_reseau",
    "uri_producteur",
    "uri_preleveur",

    # Détails sur la zone verticale prospectée (profondeur), difficulté et accréditation
    "code_zone_verticale_prospectee",
    "mnemo_zone_verticale_prospectee",
    "code_difficulte",
    "mnemo_difficulte",
    "code_accreditation",
    "mnemo_accreditation",

    # Agrément (optionnel)
    "agrement",

    # Finalité et réseau (souvent redondants ou peu exploités)
    "code_finalite",
    "libelle_finalite",
    "code_reseau",
    "nom_reseau",

    # Producteur et préleveur (souvent pas nécessaire)
    "code_producteur",
    "nom_producteur",
    "code_preleveur",
    "nom_preleveur",

    # Horaires du prélèvement (rarement utiles)
    "heure_fin",
    "heure_prelevement",

    # Commentaires libres (texte difficile à analyser)
    "commentaires"
]

# Supprimer ces colonnes dans operation_pc (si elles existent)
operation_pc_clean = operation_pc.drop(columns=[col for col in colonnes_inutiles_operation_pc if col in operation_pc.columns])

In [21]:
prop_analyse_pc = colonnes_propres(analyse_pc, 'analyse_pc')


Colonnes propres à analyse_pc (66):
 - code_support
 - libelle_support
 - uri_support
 - code_fraction
 - libelle_fraction
 - uri_fraction
 - date_prelevement
 - heure_prelevement
 - date_maj_analyse
 - heure_analyse
 - code_parametre
 - libelle_parametre
 - uri_parametre
 - resultat
 - code_groupe_parametre
 - libelle_groupe_parametre
 - uri_groupe_parametre
 - code_unite
 - symbole_unite
 - uri_unite
 - code_remarque
 - mnemo_remarque
 - code_insitu
 - libelle_insitu
 - code_difficulte_analyse
 - mnemo_difficulte_analyse
 - limite_detection
 - limite_quantification
 - limite_saturation
 - incertitude_analytique
 - code_methode_fractionnement
 - nom_methode_fractionnement
 - uri_methode_fractionnement
 - code_methode_analyse
 - nom_methode_analyse
 - uri_methode_analyse
 - rendement_extraction
 - code_methode_extraction
 - nom_methode_extraction
 - uri_methode_extraction
 - code_accreditation
 - mnemo_accreditation
 - agrement
 - code_statut
 - mnemo_statut
 - code_qualification
 - l

In [22]:
colonnes_inutiles_analyse_pc = [
    # URIs (généralement non nécessaires en analyse classique)
    "uri_support",
    "uri_fraction",
    "uri_parametre",
    "uri_groupe_parametre",
    "uri_unite",
    "uri_methode_fractionnement",
    "uri_methode_analyse",
    "uri_methode_extraction",
    "uri_accreditation",
    "uri_reseau",
    "uri_producteur_prelevement",
    "uri_preleveur",
    "uri_laboratoire",
    
    # Infos méthode / extraction / fractionnement détaillées, souvent non critiques
    "code_methode_fractionnement",
    "nom_methode_fractionnement",
    "code_methode_analyse",
    "nom_methode_analyse",
    "rendement_extraction",
    "code_methode_extraction",
    "nom_methode_extraction",
    
    # Infos sur accréditation, agrément et qualification souvent peu utiles
    "code_accreditation",
    "mnemo_accreditation",
    "agrement",
    "code_qualification",
    "libelle_qualification",
    
    # Statut et remarques, généralement peu exploitables automatiquement
    "code_statut",
    "mnemo_statut",
    "code_remarque",
    "mnemo_remarque",
    "commentaires_analyse",
    "commentaires_resultat_analyse",
    
    # Infos réseaux, producteurs, laboratoires, préleveurs (souvent peu exploité)
    "code_reseau",
    "nom_reseau",
    "code_producteur_analyse",
    "nom_producteur_analyse",
    "code_preleveur",
    "nom_preleveur",
    "code_laboratoire",
    "nom_laboratoire",
    
    # Infos redondantes avec autres tables ou détails techniques non essentiels
    "heure_prelevement",
    "heure_analyse",
    "code_insitu",
    "libelle_insitu",
    "code_difficulte_analyse",
    "mnemo_difficulte_analyse",
    "code_point_eau_surface",
    "code_banque_reference"
]

# Supprimer ces colonnes dans analyse_pc (si elles existent)
analyse_pc_clean = analyse_pc.drop(columns=[col for col in colonnes_inutiles_analyse_pc if col in analyse_pc.columns])

In [23]:
prop_cond_env_pc = colonnes_propres(cond_env_pc, 'cond_env_pc')


Colonnes propres à cond_env_pc (38):
 - date_prelevement
 - code_parametre
 - libelle_parametre
 - uri_parametre
 - resultat
 - code_unite
 - symbole_unite
 - uri_unite
 - code_remarque
 - code_statut
 - mnemo_statut
 - code_qualification
 - libelle_qualification
 - commentaire
 - date_mesure
 - heure_mesure
 - code_methode
 - nom_methode
 - uri_methode
 - code_producteur
 - nom_producteur
 - uri_producteur
 - code_preleveur
 - nom_preleveur
 - uri_preleveur
 - libelle_resultat
 - mnemo_remarque
 - code_groupe_parametre
 - libelle_groupe_parametre
 - code_masse_deau
 - code_banque_reference
 - code_point_eau_surface
 - code_prelevement
 - date_maj
 - uri_groupe_parametre
 - code_eu_masse_deau
 - code_operation_cep
 - nom_masse_deau


In [24]:
colonnes_inutiles_cond_env_pc = [
    # URIs (souvent non utiles en analyse classique)
    "uri_parametre",
    "uri_unite",
    "uri_methode",
    "uri_producteur",
    "uri_preleveur",
    "uri_groupe_parametre",

    # Codes et libellés liés à qualification, statut, remarques (souvent peu exploités)
    "code_remarque",
    "code_statut",
    "mnemo_statut",
    "code_qualification",
    "libelle_qualification",
    "mnemo_remarque",

    # Infos sur producteurs, préleveurs (souvent non indispensables)
    "code_producteur",
    "nom_producteur",
    "code_preleveur",
    "nom_preleveur",

    # Détails sur banques de référence, points d'eau surface
    "code_banque_reference",
    "code_point_eau_surface",

    # Autres codes techniques souvent redondants ou peu utilisés
    "code_masse_deau",
    "code_eu_masse_deau",
    "code_operation_cep",

    # Dates et heures peu exploitables (choisir la plus pertinente entre date_prelevement et date_mesure)
    "heure_mesure",
    "date_maj",
    "date_mesure"
]

# Supprimer ces colonnes dans cond_env_pc (si elles existent)
cond_env_pc_clean = cond_env_pc.drop(columns=[col for col in colonnes_inutiles_cond_env_pc if col in cond_env_pc.columns])

In [25]:
print("=== station_pc ===")
print(f"Taille avant nettoyage : {station_pc.shape}")
print(f"Taille après nettoyage  : {station_pc_clean.shape}\n")

print("=== operation_pc ===")
print(f"Taille avant nettoyage : {operation_pc.shape}")
print(f"Taille après nettoyage  : {operation_pc_clean.shape}\n")

print("=== analyse_pc ===")
print(f"Taille avant nettoyage : {analyse_pc.shape}")
print(f"Taille après nettoyage  : {analyse_pc_clean.shape}\n")

print("=== cond_env_pc ===")
print(f"Taille avant nettoyage : {cond_env_pc.shape}")
print(f"Taille après nettoyage  : {cond_env_pc_clean.shape}\n")


=== station_pc ===
Taille avant nettoyage : (4, 42)
Taille après nettoyage  : (4, 23)

=== operation_pc ===
Taille avant nettoyage : (3718, 42)
Taille après nettoyage  : (3718, 15)

=== analyse_pc ===
Taille avant nettoyage : (400748, 70)
Taille après nettoyage  : (400748, 24)

=== cond_env_pc ===
Taille avant nettoyage : (10932, 42)
Taille après nettoyage  : (10932, 18)



Affichage des métadonnées conservées

In [26]:
import pandas as pd

def afficher_par_tranches(df, taille_tranche=20):
    total_colonnes = df.shape[1]
    for i in range(0, total_colonnes, taille_tranche):
        fin = min(i + taille_tranche, total_colonnes)
        print(f"\nColonnes {i+1} à {fin} sur {total_colonnes} :")
        display(df.iloc[:, i:fin])  # Utilise display() si dans un notebook Jupyter, sinon print()



afficher_par_tranches(cond_env_pc_clean)



Colonnes 1 à 18 sur 18 :


,code_station,libelle_station,date_prelevement,code_parametre,libelle_parametre,resultat,code_unite,symbole_unite,commentaire,code_methode,nom_methode,libelle_resultat,code_groupe_parametre,libelle_groupe_parametre,code_prelevement,nom_masse_deau,longitude,latitude
0,3001000,LA SEINE A NOD-SUR-SEINE 1,2007-02-05,1429,Cote à l'échelle,0.98,111,m,Producteur : Lieu Analyse:PARAMETRE TERRAIN/Ec...,NaN,NaN,0.98,"['73', '34', '32', '31', '213', '209', '165', ...","['Paramètres classés par usage', 'Environnemen...",20190316033133-PC-94644,la Seine du confluent du Brevon exclu au con...,4.570986,47.764194
1,3001000,LA SEINE A NOD-SUR-SEINE 1,2007-02-05,1409,Température de l'air,7.00,27,°C,Producteur : Lieu Analyse:PARAMETRE TERRAIN/Th...,NaN,NaN,7,"['73', '34', '32', '31']","['Paramètres classés par usage', 'Environnemen...",20190316033133-PC-94644,la Seine du confluent du Brevon exclu au con...,4.570986,47.764194
2,3001000,LA SEINE A NOD-SUR-SEINE 1,2007-02-26,1429,Cote à l'échelle,0.84,111,m,Producteur : Lieu Analyse:PARAMETRE TERRAIN/Ec...,NaN,NaN,0.84,"['73', '34', '32', '31', '213', '209', '165', ...","['Paramètres classés par usage', 'Environnemen...",20190315213401-PC-60366,la Seine du confluent du Brevon exclu au con...,4.570986,47.764194
3,3001000,LA SEINE A NOD-SUR-SEINE 1,2007-02-26,1429,Cote à l'échelle,0.84,111,m,Producteur : Lieu Analyse:PARAMETRE TERRAIN/Ec...,NaN,NaN,0.84,"['73', '34', '32', '31', '213', '209', '165', ...","['Paramètres classés par usage', 'Environnemen...",20190315213401-PC-60366,la Seine du confluent du Brevon exclu au con...,4.570986,47.764194
4,3001000,LA SEINE A NOD-SUR-SEINE 1,2007-02-26,1409,Température de l'air,8.00,27,°C,Producteur : Lieu Analyse:PARAMETRE TERRAIN/Th...,NaN,NaN,8,"['73', '34', '32', '31']","['Paramètres classés par usage', 'Environnemen...",20190315213401-PC-60366,la Seine du confluent du Brevon exclu au con...,4.570986,47.764194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10927,3174000,LA SEINE A POSES 2,2025-05-01,1947,Type de prélèvement,NaN,0,Unité inconnue,NaN,0.0,Méthode inconnue,NaN,"['73', '34', '32', '31']","['Paramètres classés par usage', 'Environnemen...",20250418095700-PC-594238,La Seine du confluent de l Epte inclus au co...,1.236656,49.310144
10928,3174000,LA SEINE A POSES 2,2025-05-01,1726,Situation hydrologique apparente,NaN,0,Unité inconnue,NaN,0.0,Méthode inconnue,NaN,"['73', '34', '32', '31']","['Paramètres classés par usage', 'Environnemen...",20250418095700-PC-594238,La Seine du confluent de l Epte inclus au co...,1.236656,49.310144
10929,3174000,LA SEINE A POSES 2,2025-05-01,1422,Limpidité de l'eau,NaN,0,Unité inconnue,NaN,0.0,Méthode inconnue,NaN,"['73', '34', '32', '31']","['Paramètres classés par usage', 'Environnemen...",20250418095700-PC-594238,La Seine du confluent de l Epte inclus au co...,1.236656,49.310144
10930,3174000,LA SEINE A POSES 2,2025-05-01,1413,Présence de produits ligneux ou herbacés frais,NaN,0,Unité inconnue,NaN,0.0,Méthode inconnue,NaN,"['73', '38', '37', '34', '31']","['Paramètres classés par usage', 'Hydrobiologi...",20250131094133-PC-589894,La Seine du confluent de l Epte inclus au co...,1.236656,49.310144


stats_clean_a_supp: pk_aval, localisation_precise_station, objectifs_operation, libelle_support, code_support, code_occupation_sol,	libelle_occupation_sol,	,zone_huet	

ops_clean_a_supp : etat_avancement_operation, debit_journalier, presence_secteur_debit_reserve, presence_secteur_eclusee, presence_secteur_soutien_etiage, observations_hydrologie, profondeur, sinuosite, nombre_traits, nombre_points_anguilles, protocole_peche, moyen_prospection, objectifs_operation, operateur_code, operateur_libelle, operateur_libelle_aspe, temperature_max_eau,	code_categorie_piscicole, libelle_categorie_piscicole	,temperature_air_bassin_versant, precipitation_bassin_versant, amplitude_thermique_air_station, temperature_air_station, duree_peche,	surface_eq_radier_rapide_troncon,longueur,	largeur_lame_eau,	longueur_rive_droite, largeur_rive_droite, longueur_rive_gauche, largeur_rive_gauche	,surface_calculee, temperature_instantanee, conductivite, debit_journalier,	presence_secteur_debit_reserve,	presence_secteur_eclusee,	,presence_secteur_soutien_etiage,	observations_hydrologie,profondeur, sinuosite	

obs_cleana_supp : objectifs_operation, code_prelevement_elementaire	type_prelevement_elementaire	duree_prelevement_elementaire	type_points	nombre_points	nombre_points_sans_poisson	nom_ambiance	situation_ambiance	localisation_ambiance	code_ombrage_ambiance	libelle_ombrage_ambiance	code_granulo_ambiance_dominante	libelle_granulo_ambiance_dominante	vegetation_ambiance	abondance_cache_vegetation_aquatique_ambiance	abondance_cache_embacles_ambiance	abondance_cache_vegetation_bordure_ambiance	abondance_cache_sous_berge_ambiance	observation_ambiance

indic_cleana_supp : iprplus_uris_taxon, objectifs_operation ipr_surface_prospectee	ipr_largeur_eau	ipr_profondeur	ipr_temperature_air_janvier	ipr_temperature_air_juillet ipr_uris_taxon iprplus_strategie_echantillonnage	iprplus_largeur_eau	 iprplus_uris_taxon

station_pc_cleana_supp : 	altitude_point_caracteristique	point_kilometrique	superficie_bassin_versant_reel durete 

operation_pc_cleana_supp : code_support	libelle_support	code_methode, nom_methode, date_fin,	profondeur,	code_point_eau_surface,	code_banque_reference

analyse_pc_cleana_supp : longitude, code_support	libelle_support code_fraction	libelle_fraction date_maj_analyse	

cond_env_pc_cleana_supp : commentaire	code_methode	nom_methode

In [27]:
stats_clean_a_supp = [
    "pk_aval", "localisation_precise_station", "objectifs_operation",
    "libelle_support", "code_support", "code_occupation_sol",
    "libelle_occupation_sol", "zone_huet"
]

ops_clean_a_supp = [
    "etat_avancement_operation", "debit_journalier", "presence_secteur_debit_reserve",
    "presence_secteur_eclusee", "presence_secteur_soutien_etiage",
    "observations_hydrologie", "profondeur", "sinuosite", "nombre_traits",
    "nombre_points_anguilles", "protocole_peche", "moyen_prospection",
    "objectifs_operation", "operateur_code", "operateur_libelle",
    "operateur_libelle_aspe", "temperature_max_eau", "code_categorie_piscicole",
    "libelle_categorie_piscicole", "temperature_air_bassin_versant",
    "precipitation_bassin_versant", "amplitude_thermique_air_station",
    "temperature_air_station", "duree_peche", "surface_eq_radier_rapide_troncon",
    "longueur", "largeur_lame_eau", "longueur_rive_droite",
    "largeur_rive_droite", "longueur_rive_gauche", "largeur_rive_gauche",
    "surface_calculee", "temperature_instantanee", "conductivite"
]

obs_clean_a_supp = [
    "objectifs_operation", "code_prelevement_elementaire", "type_prelevement_elementaire",
    "duree_prelevement_elementaire", "type_points", "nombre_points",
    "nombre_points_sans_poisson", "nom_ambiance", "situation_ambiance",
    "localisation_ambiance", "code_ombrage_ambiance", "libelle_ombrage_ambiance",
    "code_granulo_ambiance_dominante", "libelle_granulo_ambiance_dominante",
    "vegetation_ambiance", "abondance_cache_vegetation_aquatique_ambiance",
    "abondance_cache_embacles_ambiance", "abondance_cache_vegetation_bordure_ambiance",
    "abondance_cache_sous_berge_ambiance", "observation_ambiance"
]

indic_clean_a_supp = [
    "iprplus_uris_taxon", "objectifs_operation", "ipr_surface_prospectee",
    "ipr_largeur_eau", "ipr_profondeur", "ipr_temperature_air_janvier",
    "ipr_temperature_air_juillet", "ipr_uris_taxon", "iprplus_strategie_echantillonnage",
    "iprplus_largeur_eau"
]

station_pc_clean_a_supp = [
    "altitude_point_caracteristique", "point_kilometrique",
    "superficie_bassin_versant_reel", "durete"
]

operation_pc_clean_a_supp = [
    "code_support", "libelle_support", "code_methode", "nom_methode",
    "date_fin", "profondeur", "code_point_eau_surface", "code_banque_reference"
]

analyse_pc_clean_a_supp = [
    "longitude", "code_support", "libelle_support", "code_fraction",
    "libelle_fraction", "date_maj_analyse"
]

cond_env_pc_clean_a_supp = [
    "commentaire", "code_methode", "nom_methode"
]


In [28]:
# Suppression des colonnes si elles existent
stats_clean_final = stats_clean.drop(columns=[col for col in stats_clean_a_supp if col in stats_clean.columns])
ops_clean_final = ops_clean.drop(columns=[col for col in ops_clean_a_supp if col in ops_clean.columns])
obs_clean_final = obs_clean.drop(columns=[col for col in obs_clean_a_supp if col in obs_clean.columns])
indic_clean_final = indic_clean.drop(columns=[col for col in indic_clean_a_supp if col in indic_clean.columns])
station_pc_clean_final = station_pc_clean.drop(columns=[col for col in station_pc_clean_a_supp if col in station_pc_clean.columns])
operation_pc_clean_final = operation_pc_clean.drop(columns=[col for col in operation_pc_clean_a_supp if col in operation_pc_clean.columns])
analyse_pc_clean_final = analyse_pc_clean.drop(columns=[col for col in analyse_pc_clean_a_supp if col in analyse_pc_clean.columns])
cond_env_pc_clean_final = cond_env_pc_clean.drop(columns=[col for col in cond_env_pc_clean_a_supp if col in cond_env_pc_clean.columns])

In [29]:
# Affichage de la structure des DataFrames poissons

print("=== poissons_indicateurs ===")
print(f"Taille avant nettoyage : {indic_clean.shape}")
print(f"Taille après nettoyage  : {indic_clean_final.shape}\n")

print("=== poissons_observations ===")
print(f"Taille avant nettoyage : {obs_clean.shape}")
print(f"Taille après nettoyage  : {obs_clean_final.shape}\n")

print("=== poissons_operations ===")
print(f"Taille avant nettoyage : {ops_clean.shape}")
print(f"Taille après nettoyage  : {ops_clean_final.shape}\n")

print("=== poissons_stations ===")
print(f"Taille avant nettoyage : {stats_clean.shape}")
print(f"Taille après nettoyage  : {stats_clean_final.shape}\n")

# Affichage de la structure des DataFrames analyse des cours d'eau

print("=== station_pc ===")
print(f"Taille avant nettoyage : {station_pc_clean.shape}")
print(f"Taille après nettoyage  : {station_pc_clean_final.shape}\n")

print("=== operation_pc ===")
print(f"Taille avant nettoyage : {operation_pc_clean.shape}")
print(f"Taille après nettoyage  : {operation_pc_clean_final.shape}\n")

print("=== analyse_pc ===")
print(f"Taille avant nettoyage : {analyse_pc_clean.shape}")
print(f"Taille après nettoyage  : {analyse_pc_clean_final.shape}\n")

print("=== cond_env_pc ===")
print(f"Taille avant nettoyage : {cond_env_pc_clean.shape}")
print(f"Taille après nettoyage  : {cond_env_pc_clean_final.shape}\n")


=== poissons_indicateurs ===
Taille avant nettoyage : (352, 80)
Taille après nettoyage  : (352, 70)

=== poissons_observations ===
Taille avant nettoyage : (80918, 72)
Taille après nettoyage  : (80918, 52)

=== poissons_operations ===
Taille avant nettoyage : (489, 69)
Taille après nettoyage  : (489, 35)

=== poissons_stations ===
Taille avant nettoyage : (39, 32)
Taille après nettoyage  : (39, 27)

=== station_pc ===
Taille avant nettoyage : (4, 23)
Taille après nettoyage  : (4, 19)

=== operation_pc ===
Taille avant nettoyage : (3718, 15)
Taille après nettoyage  : (3718, 7)

=== analyse_pc ===
Taille avant nettoyage : (400748, 24)
Taille après nettoyage  : (400748, 18)

=== cond_env_pc ===
Taille avant nettoyage : (10932, 18)
Taille après nettoyage  : (10932, 15)



In [34]:
clean_folder = os.path.abspath(os.path.join(os.getcwd(), "..", f"{riviere}_donnees_cleaned"))
subfolders = ["poissons", "qualite_rivieres"]

# Création des dossiers
for sub in subfolders:
    os.makedirs(os.path.join(clean_folder, sub), exist_ok=True)

def path_csv(subfolder, filename):
    # Ici on garde la majuscule dans le préfixe (ex: Seine_poissons_indicateurs.csv)
    return os.path.join(clean_folder, subfolder, f"{riviere}_{filename}.csv")

# Sauvegarde fichiers poissons
indic_clean_final.to_csv(path_csv("poissons", "poissons_indicateurs"), index=False)
obs_clean_final.to_csv(path_csv("poissons", "poissons_observations"), index=False)
ops_clean_final.to_csv(path_csv("poissons", "poissons_operations"), index=False)
stats_clean_final.to_csv(path_csv("poissons", "poissons_stations"), index=False)

# Sauvegarde fichiers qualité des rivières
station_pc_clean_final.to_csv(path_csv("qualite_rivieres", "station_pc"), index=False)
operation_pc_clean_final.to_csv(path_csv("qualite_rivieres", "operation_pc"), index=False)
analyse_pc_clean_final.to_csv(path_csv("qualite_rivieres", "analyse_pc"), index=False)
cond_env_pc_clean_final.to_csv(path_csv("qualite_rivieres", "cond_env_pc"), index=False)

# Création du zip
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(clean_folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, clean_folder))

print(f"Les données nettoyées ont été sauvegardées dans {zip_path}")

# Suppression du dossier temporaire
shutil.rmtree(clean_folder)
print(f"Le dossier temporaire '{clean_folder}' a été supprimé.")


Les données nettoyées ont été sauvegardées dans C:\Users\ferna\Desktop\API-Hub-Eau\Seine_donnees.zip
Le dossier temporaire 'C:\Users\ferna\Desktop\API-Hub-Eau\Seine_donnees_cleaned' a été supprimé.
